In [164]:
import datetime

import pandas as pd
import numpy as np
import sklearn 
import pydot

from sklearn import tree
from sklearn.tree import export_graphviz
from sklearn.tree import DecisionTreeClassifier
from sklearn import model_selection

In [165]:
#read in
names = ['State-FIPS_Code', 'County_FIPS_Code', 'Strata_ID_Number', 'Number_Counties', 'Population_Size']    

demographics = pd.read_csv("data/DEMOGRAPHICS.csv", names=names)
summary = pd.read_csv("data/SUMMARYMEASURESOFHEALTH.csv", names=names)
leading = pd.read_csv("data/LEADINGCAUSESOFDEATH.csv", names=names)
measure = pd.read_csv("data/MEASURESOFBIRTHANDDEATH.csv", names=names)
vunerable  = pd.read_csv("data/VUNERABLEPOPSANDENVHEALTH.csv", names=names)
risk = pd.read_csv("data/RISKFACTORSANDACCESSTOCARE.csv", names=names)

In [166]:
#merge
full = pd.concat([demographics, summary,leading, measure, vunerable,risk], axis=1, join="inner")

In [167]:
full.describe()

,State-FIPS_Code,County_FIPS_Code,Strata_ID_Number,Number_Counties,Population_Size,State-FIPS_Code,County_FIPS_Code,Strata_ID_Number,Number_Counties,Population_Size,...,State-FIPS_Code,County_FIPS_Code,Strata_ID_Number,Number_Counties,Population_Size,State-FIPS_Code,County_FIPS_Code,Strata_ID_Number,Number_Counties,Population_Size
count,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
unique,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [171]:
def _main():
	# load dataset
	dataset = summary    
    
	array = dataset.values
	X = array[:,0:4]
	Y = array[:,4]
	test_size = 0.20
	seed = 7
	X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, Y, test_size=test_size, random_state=seed)

	# test options and perform evaluation metric
	seed = 7
	scoring = 'accuracy'
	models = []
	models.append(('DecisionTree', DecisionTreeClassifier()))


	dtc = tree.DecisionTreeClassifier()
	_train(dtc, "Decision Tree Classifier", X_train, Y_train, X_test, Y_test)
    
	with open('DecisionTreeClassifier.dot', 'w') as file:
		tree.export_graphviz(dtc, out_file = file)
	file.close()
    
	(graph,) = pydot.graph_from_dot_file('DecisionTreeClassifier.dot')
	graph.write_png('DecisionTreeClassifier.png')    
    
    
	models.clear()
	models.append(('DecisionTree', dtc))
	_predictionAccuracySummary(models, X_train, Y_train, X_test, Y_test)
    
    
def _train(alg,algName, X_train, Y_train, X_test, Y_test):
	print(datetime.datetime.now(), "Begin training: ", algName)
	alg.fit(X_train, Y_train)
	print(datetime.datetime.now(), "End training: ", algName)    

def _predictionAccuracySummary(models, X_train, Y_train, X_test, Y_test):
	print("Algorithm\t\tAccuracy Score")
	for name, model in models:
		predictions = model.predict(X_test)
		print("%20s\t\t%0.2f" % (name, accuracy_score(Y_test, predictions)))
	
_main()

2019-03-13 20:34:25.625665 Begin training:  Decision Tree Classifier
2019-03-13 20:34:25.637658 End training:  Decision Tree Classifier
Algorithm		Accuracy Score


ValueError: could not convert string to float: 'Min_Unhealthy_Days'